<a href="https://colab.research.google.com/github/AnyaAP/ComputerGraphics/blob/main/z_buffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pygame
from pygame.locals import *

# Инициализация Pygame
pygame.init()

# Размеры экрана
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Z-Buffer')

# Значение бесконечно удаленного Z (очень большое значение)
far_z = 1000

# Создание Z-буфера и заполнение его бесконечными значениями
z_buffer = [[far_z for _ in range(width)] for __ in range(height)]

# Функция для рисования треугольника с использованием Z-буфера
def draw_triangle(vertices):
    # Извлечение координат вершин треугольника
    v0, v1, v2 = vertices

    # Определение границ для сканирования по Y
    y_min = min(v0[1], v1[1], v2[1])
    y_max = max(v0[1], v1[1], v2[1])

    for y in range(y_min, y_max + 1):
        # Нахождение X-координат на линиях между вершинами
        x0, x1, x2 = interpolate(v0, v1, v2, y)

        # Определение минимального и максимального X для сканирования по X
        x_min = min(x0, x1, x2)
        x_max = max(x0, x1, x2)

        for x in range(x_min, x_max + 1):
            if is_inside_triangle(x, y, v0, v1, v2):
                # Расчет глубины пикселя в текущих координатах (Z-координата)
                z = calculate_depth(v0, v1, v2, x, y)

                # Проверка, виден ли пиксель
                if z < z_buffer[y][x]:
                    z_buffer[y][x] = z
                    screen.set_at((x, y), (255, 255, 255))  # Отрисовка пикселя

# Функция для линейной интерполяции X-координат
def interpolate(v0, v1, v2, y):
    x0 = v0[0] + (v2[0] - v0[0]) * (y - v0[1]) / (v2[1] - v0[1])
    x1 = v0[0] + (v1[0] - v0[0]) * (y - v0[1]) / (v1[1] - v0[1])
    x2 = v1[0] + (v2[0] - v1[0]) * (y - v1[1]) / (v2[1] - v1[1])
    return int(x0), int(x1), int(x2)

# Функция для проверки, находится ли точка внутри треугольника
def is_inside_triangle(x, y, v0, v1, v2):
    # Вычисление барицентрических координат
    b0 = ((v1[1] - v2[1]) * (x - v2[0]) + (v2[0] - v1[0]) * (y - v2[1])) / \
         ((v1[1] - v2[1]) * (v0[0] - v2[0]) + (v2[0] - v1[0]) * (v0[1] - v2[1]))
    b1 = ((v2[1] - v0[1]) * (x - v2[0]) + (v0[0] - v2[0]) * (y - v2[1])) / \
         ((v1[1] - v2[1]) * (v0[0] - v2[0]) + (v2[0] - v1[0]) * (v0[1] - v2[1]))
    b2 = 1 - b0 - b1

    # Проверка, находится ли точка внутри треугольника
    return 0 <= b0 <= 1 and 0 <= b1 <= 1 and 0 <= b2 <= 1

# Функция для вычисления глубины (Z-координаты) пикселя
def calculate_depth(v0, v1, v2, x, y):
    # Вычисление барицентрических координат для треугольника
    b0 = ((v1[1] - v2[1]) * (x - v2[0]) + (v2[0] - v1[0]) * (y - v2[1])) / \
         ((v1[1] - v2[1]) * (v0[0] - v2[0]) + (v2[0] - v1[0]) * (v0[1] - v2[1]))
    b1 = ((v2[1] - v0[1]) * (x - v2[0]) + (v0[0] - v2[0]) * (y - v2[1])) / \
         ((v1[1] - v2[1]) * (v0[0] - v2[0]) + (v2[0] - v1[0]) * (v0[1] - v2[1]))
    b2 = 1 - b0 - b1

    # Вычисление глубины (Z-координаты)
    depth = b0 * v0[2] + b1 * v1[2] + b2 * v2[2]
    return depth

# Пример трех вершин треугольника (x, y, z)
vertices = [
    (200, 100, 20),
    (400, 500, 40),
    (600, 200, 30)
]

# Рисование треугольника
draw_triangle(vertices)

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
